In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
wos_train = pd.read_csv("/opt/CHAN-DST/data/multiwos/ra_train.tsv",sep='\t')
wos_test = pd.read_csv("/opt/CHAN-DST/data/multiwos/ra_test.tsv",sep='\t')
woz2_train = pd.read_csv("/opt/CHAN-DST/data/multiwoz/train.tsv",sep='\t')
woz2_dev = pd.read_csv("/opt/CHAN-DST/data/multiwoz/dev.tsv",sep='\t')
woz2_test = pd.read_csv("/opt/CHAN-DST/data/multiwoz/test.tsv",sep='\t')

In [3]:
print(wos_train.columns[:4])
print(woz2_train.columns[:4])

Index(['# Dialogue ID', 'Turn Index', 'Last Turn', 'System Response'], dtype='object')
Index(['# Dialogue ID', 'Turn Index', 'User Utterance', 'System Response'], dtype='object')


In [4]:
new_wos_train = wos_train.drop(['Last Turn'],axis=1)
new_wos_test = wos_test.drop(['Last Turn'],axis=1)

In [5]:
print(new_wos_train.columns[:4])
print(woz2_train.columns[:4])

Index(['# Dialogue ID', 'Turn Index', 'System Response', 'User Utterance'], dtype='object')
Index(['# Dialogue ID', 'Turn Index', 'User Utterance', 'System Response'], dtype='object')


In [6]:
new_ordered_columns = new_wos_train.columns.tolist()
new_ordered_columns.pop(2)
new_ordered_columns.insert(3,'System Response')
print(new_ordered_columns[:4])

['# Dialogue ID', 'Turn Index', 'User Utterance', 'System Response']


In [7]:
new_wos_train = new_wos_train[new_ordered_columns]
new_wos_test = new_wos_test[new_ordered_columns]

new_wos_train.to_csv('/opt/CHAN-DST/data/multiwos/origin_train.tsv', index=False, header=True, sep="\t")
new_wos_test.to_csv('/opt/CHAN-DST/data/multiwos/test.tsv', index=False, header=True, sep="\t")


## train을 train과 dev로 나누기

In [8]:
wos_train = pd.read_csv('/opt/CHAN-DST/data/multiwos/origin_train.tsv', header=0, sep="\t")
wos_test = pd.read_csv('/opt/CHAN-DST/data/multiwos/test.tsv', header=0, sep="\t")
wos_train.head(3)

,# Dialogue ID,Turn Index,User Utterance,System Response,관광-경치 좋은,관광-교육적,관광-도보 가능,관광-문화 예술,관광-역사적,관광-이름,...,식당-지역,식당-흡연 가능,지하철-도착지,지하철-출발 시간,지하철-출발지,택시-도착 시간,택시-도착지,택시-종류,택시-출발 시간,택시-출발지
0,snowy-hat-8324:관광_식당_11,0,서울 중앙에 있는 박물관을 찾아주세요,NaN,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
1,snowy-hat-8324:관광_식당_11,1,좋네요 거기 평점은 말해주셨구 전화번호가 어떻게되나요?,안녕하세요. 문화역서울 284은 어떠신가요? 평점도 4점으로 방문객들에게 좋은 평가...,none,none,none,none,none,문화역서울 284,...,none,none,none,none,none,none,none,none,none,none
2,snowy-hat-8324:관광_식당_11,2,네 관광지와 같은 지역의 한식당을 가고싶은데요 야외석이 있어야되요,전화번호는 983880764입니다. 더 필요하신 게 있으실까요?,none,none,none,none,none,문화역서울 284,...,서울 중앙,none,none,none,none,none,none,none,none,none


In [9]:
import re
def extract_domain(x):
    pat = re.compile(rf'[:][가-힣_]+')
    s = re.search(pat,x).group()[1:-1]
    return s

# 순서유지하면서, 중복제거
from collections import OrderedDict
dialog_id=list(OrderedDict.fromkeys(wos_train['# Dialogue ID']))
dialog_id = pd.DataFrame(dialog_id,columns=['dialog_id'])
dialog_id['domain'] = dialog_id['dialog_id'].apply(extract_domain)

from sklearn.model_selection import StratifiedShuffleSplit

SSSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_idx, dev_idx in SSSplit.split(dialog_id, dialog_id['domain']):
    train_dialog_id = dialog_id['dialog_id'][train_idx]
    dev_dialog_id = dialog_id['dialog_id'][dev_idx]
print(len(train_dialog_id))
print(len(dev_dialog_id))

6300
700


In [10]:
from tqdm import tqdm
train=pd.DataFrame(columns=wos_train.columns)
for dialog_id in tqdm(list(train_dialog_id)):
    train=pd.concat([train,wos_train.loc[wos_train['# Dialogue ID']==dialog_id]],axis=0)
train.index=range(train.shape[0])
print(train.shape)

dev=pd.DataFrame(columns=wos_train.columns)
for dialog_id in tqdm(list(dev_dialog_id)):
    dev=pd.concat([dev,wos_train.loc[wos_train['# Dialogue ID']==dialog_id]],axis=0)
dev.index=range(dev.shape[0])
print(dev.shape)
print('='*30)
print(train.shape[0]+dev.shape[0])
print(wos_train.shape[0])

  3%|▎         | 20/700 [00:00<00:03, 194.17it/s]

(52375, 49)


100%|██████████| 700/700 [00:04<00:00, 144.25it/s]

(5870, 49)
58245
58245


In [11]:
train['# Dialogue ID'] = train['# Dialogue ID'].apply(lambda x: x.replace('-','_'))
dev['# Dialogue ID'] = dev['# Dialogue ID'].apply(lambda x: x.replace('-','_'))
wos_test['# Dialogue ID'] = wos_test['# Dialogue ID'].apply(lambda x: x.replace('-','_'))

In [15]:
train.to_csv('/opt/CHAN-DST/data/multiwos/train.tsv', index=False, header=True, sep="\t")
dev.to_csv('/opt/CHAN-DST/data/multiwos/dev.tsv', index=False, header=True, sep="\t")
wos_test.to_csv('/opt/CHAN-DST/data/multiwos/test.tsv', index=False, header=True, sep="\t")

In [10]:
train = pd.read_csv('/opt/CHAN-DST/data/multiwos/train.tsv', header=0, sep="\t")
train

,# Dialogue ID,Turn Index,User Utterance,System Response,관광-경치 좋은,관광-교육적,관광-도보 가능,관광-문화 예술,관광-역사적,관광-이름,...,식당-지역,식당-흡연 가능,지하철-도착지,지하철-출발 시간,지하철-출발지,택시-도착 시간,택시-도착지,택시-종류,택시-출발 시간,택시-출발지
0,bold-resonance-6886:숙소_관광_식당_11,0,안녕하세요? 숙소를 예약하려고 하는데요. 일단 지역은 서울 안에만 있으면 될 것 같고요.,NaN,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
1,bold-resonance-6886:숙소_관광_식당_11,1,가격대와 종류 모두 지정하지 않고 예약하겠습니다.,안녕하세요. 네. 알겠습니다. 가격대와 종류는 어떻게 설정해드리면 될까요?,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
2,bold-resonance-6886:숙소_관광_식당_11,2,아 그래요? 그럼 적당한 가격대로 해주세요.,죄송합니다만 말씀하신 대로 예약시 정보의 양이 너무 광범위해서 검색이 되지 않습니다...,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
3,bold-resonance-6886:숙소_관광_식당_11,3,아 여기 괜찮네요. 그럼 여기로 월요일부터 1일간 5명으로 예약 부탁드립니다.,알겠습니다. 서울 강남구에 위치한 평점 4점의 패밀리 호텔이 이용 가능한걸로 확인됩니다.,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
4,bold-resonance-6886:숙소_관광_식당_11,4,저 그리고 주차가 되는 관광지도 방문해보고 싶은데요. 지역이나 종류 신경쓰지 마시고...,네. 바로 예약 도와드리겠습니다.,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52370,crimson-mud-3023:관광_택시_12,2,"음... 네 좋아요. 제가 차를 놓고 중간에 택시도 좀 타야되는데요, 한가한과에서 ...",여러 곳 확인됩니다. 경복궁역쪽에 국립 고궁 박물관은 어떠실까요?,none,none,none,none,none,국립 고궁 박물관,...,none,none,none,none,none,11:30,압구정역,none,22:15,한가한과
52371,crimson-mud-3023:관광_택시_12,3,아 그럼 도착시간만 넣을게요. 11시 반까지는 꼭 가야되서...,아 죄송하지만 출발시간과 도착시간 중 하나만 정해주시겠어요? 시스템상 두 곳을 다 ...,none,none,none,none,none,국립 고궁 박물관,...,none,none,none,none,none,11:30,압구정역,none,22:15,한가한과
52372,crimson-mud-3023:관광_택시_12,4,종류요? 네 딱히 생각안해봤어요.,택시 종류는 상관없으신거죠?,none,none,none,none,none,국립 고궁 박물관,...,none,none,none,none,none,11:30,압구정역,dontcare,dontcare,한가한과
52373,crimson-mud-3023:관광_택시_12,5,다 말씀해주셨어요. 감사합니다.,네 택시 확인됩니다. 한가한과에서 10시 45분에 출발하며 요금은 만원 예상됩니다....,none,none,none,none,none,국립 고궁 박물관,...,none,none,none,none,none,11:30,압구정역,dontcare,dontcare,한가한과


In [29]:
train_bool_idx=wos_train['# Dialogue ID'].apply(lambda x: x in list(train_dialog_id))
dev_bool_idx=wos_train['# Dialogue ID'].apply(lambda x: x in list(dev_dialog_id))
print(sum(train_bool_idx))
print(sum(dev_bool_idx))

52375
5870


In [19]:
# def columns_to_domain(x):
#     check_series = x.loc['관광-경치 좋은':'택시-출발지']
#     return ','.join([c.split('-')[0] for c in check_series.index if check_series[c]!='none'])
# wos_train['domains'] = wos_train.apply(columns_to_domain, axis=1)

# def domain_union(x):
#     return set(','.join(x.domains).split(','))
# wos_train.groupby(['# Dialogue ID']).apply(domain_union)